In [2]:
import qutip, pickle, sys
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as opt 
import scipy.linalg as linalg
import time as time
import math, cmath
import Max_Ent_aux_lib as melib
#import proj_ev_library as projev
#import max_entev library as meev
from IPython.display import display, Math, Latex

np.set_printoptions(threshold=1.e-3,linewidth=120,precision=1, formatter={"float":lambda x: str(.001*int(1000*x)) })

Tenemos la siguiente ecuación de movimiento:

$$
    \dot{c}_M(t) I_{M}^{0}(t) = -i \bigg\{\bigg(c_{M+2}(t) - c_M(t) \bigg)\langle \rho_{M}^{0\dagger}(t) [H_{0,-2}, \rho_{M+2}^0] \rangle + \bigg(c_{M-2}(t) - c_M(t)\bigg) \langle \rho_{M}^{0\dagger}(t) [H_{0,2}, \rho_{M-2}^0] \rangle + p \langle \rho_{M}^{0\dagger}(t) [\Sigma, \rho_{M}^0 (t)]\rangle \bigg\}
$$

siendo $I_{M}^{0}(t) = \langle \rho_{M}^{0\dagger}(t) \rho_{M}^{0}(t)\rangle = I_{-M}^{0}(t)$
la cual puede reescribirse como 

$$
\dot{c}_M(t) I_{M}^{0}(t) = -i \bigg\{A_{\rho^{M, M+2}} c_{M+2}(t) - \bigg(A_{\rho^{M, M+2}} + B_{\rho^{M, M-2}} \bigg) c_{M}(t) + B_{\rho^{M, M-2}} c_{M-2}(t) \bigg\} + p C_{\rho^M} \\
= - i{\bf{\mathcal{M}}}(t) \textbf{c}(t)
$$

donde 

<ol>

<li> $A_{\rho^{M, M+2}} = \langle \rho_{M}^{0\dagger}(t) [H_{0,-2}, \rho_{M+2}^0] \rangle$ </li> 
<li> $B_{\rho^{M, M-2}} = \langle \rho_{M}^{0\dagger}(t) [H_{0,2}, \rho_{M-2}^0] \rangle$ </li>
<li> $C_{\rho^M} = \langle \rho_{M}^{0\dagger}(t) [\Sigma, \rho_{M}^0 (t)]\rangle$ </li>
    
</ol>    

Entonces, si $f(t) = \langle \rho(t) \rho^0(t) \rangle = \sum_{M} \dot{c}_M(t) I_{M}^{0}(t)$

$$
    \dot{f}(t) = \sum_{M} \bigg(\dot{c}_M(t) I_{M}^{0}(t) + {c}_M(t) \dot{I}_{M}^{0}(t) \bigg)
$$

In [5]:
1: A=alpha * B=N(t) * exp(m * t**(-a)) = I_M**(0) (a primer orden alpha = 1) 
2: hacer la matriz hermìtica
3: C = M * N(t) * exp(m * t**(-a))
    
K(t,t') = e**(-i M(t-t')) 
    
con N(t) = 1/Tr(rho(t)) la calculo con la definición del rho_M = sum_{M} c_M(t) 

SyntaxError: invalid syntax (1270290635.py, line 1)

In [46]:
def norm_at_timet(rho_M, time_t_minus_one):
    return sum(1)

def A_mmplustwo_matrix_elmt(cohrnc, time, power_law_factor):
    return np.e**(cohrnc * time**(-power_law_factor))

def B_mmminustwo_matrix_elmt(cohrnc, time, power_law_factor):
    return np.e**(cohrnc * time**(-power_law_factor))

def C_m_matrix_elmt(cohrnc, time, power_law_factor):
    return cohrnc * np.e**(cohrnc * time**(-power_law_factor))

def diag_mm_matrix_elmt(cohr, time, power_law_factor):
    return (-A_mmplustwo_matrix_elmt(cohr, time, power_law_factor) - B_mmminustwo_matrix_elmt(cohr, time, power_law_factor) 
            + p * C_m_matrix_elmt(cohr, time, power_law_factor))

def generating_function_M_matrix(parameters, cmt0_coeff_list, time, cbc = False):
    M = parameters["total_no_cohrs"]; p = parameters["p_factor"];  a = parameters["power_law_factor"]
    cm_list = cmt0_coeff_list
    m_matrix_list = []; t = time;
    
    for m in range(M):
        if m == 0:
            m_matrix_list.append(np.array([diag_mm_matrix_elmt(m,t,a)] + [0]
                                    + [A_mmplustwo_matrix_elmt(m+2,t,a)] + [0 for k in range(M-3)])) 
        if m == 1:
            local_list = [0] + [diag_mm_matrix_elmt(m,t,a)] + [0] + [A_mmplustwo_matrix_elmt(m+2, t, a)]
            local_length = len(np.array(local_list))
            m_matrix_list.append(local_list + [0 for k in range(M-local_length)])
        if (m > 1) and (m < M - 2):  
            local_list = [0 for j in range(m-2)]; local_length = len(local_list)
            local_list += ([B_mmminustwo_matrix_elmt(-m-2,t,a)] + [0] + [diag_mm_matrix_elmt(m,t,a)] + [0] 
                        + [A_mmplustwo_matrix_elmt(m+2,t,a)])
            local_list += [0 for j in range(M-local_length - 5)]
            m_matrix_list.append(np.array(local_list))
        if m == M - 2: 
            m_matrix_list.append(np.array([0 for i in range(m-2)] + [B_mmminustwo_matrix_elmt(-m-2,t,a)] + [0] 
                                 + [diag_mm_matrix_elmt(m,t,a)] + [0]))
        if m == M - 1:
            m_matrix_list.append(np.array([0 for i in range(m-2)] + [B_mmminustwo_matrix_elmt(-m-2,t,a)] + [0] 
                                 + [diag_mm_matrix_elmt(m,t,a)]))
            
    return qutip.Qobj(m_matrix_list)

In [60]:
p = .5; a = 1e-5; M = 20; coherences_t0_pert0 = 1.
param_list = {"total_no_cohrs": M, "p_factor": p, "power_law_factor": a}
coeff_list_t0 = [coherences_t0_pert0 + np.random.rand() for i in range(param_list["total_no_cohrs"])]
M_tensor = np.array(generating_function_M_matrix(param_list, coeff_list_t0, 1, False))


In [61]:
qutip.Qobj(M_tensor)

Quantum object: dims = [[20], [20]], shape = (20, 20), type = oper, isherm = False
Qobj data =
[[-2.0 0.0 7.389 ... 0.0 0.0 0.0]
 [0.0 -4.077 0.0 ... 0.0 0.0 0.0]
 [0.018000000000000002 0.0 -7.389 ... 0.0 0.0 0.0]
 ...
 [0.0 0.0 0.0 ... 157007192.898 0.0 178482300.963]
 [0.0 0.0 0.0 ... 0.0 459619783.961 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 1338617257.223]]

In [ ]:
# Si la matrix M(t) = M para todo t, uso la propiedad de semigrupo y ya fue

def semigroup_phit_and_rhot_sol(coeff_list_t0, Mtensor, ts):
    coeff_at_timet_list = []
    coeff_at_timet_list.append(np.array(coeff_list_t0))
    
    for i in range(1, len(ts)-1):
        try: 
            a = (ts[i+1] * Mtensor).expm() * coeff_list_t0[0]
            coeff_at_timet_list.append(a)
        #if (Exception):
        #   continue 
    return coeff_at_timet_list    

# Si la matrix M(t) fuese no constante, odeint

from scipy.integrate import ode
import random

def F(y, t, Mtensor):
    return Mtensor.dot(y)

real_Mtensor = (np.real(Mtensor))
ts1 = np.linspace(0, 500, 200)
result = odeint(func=F, y0=coeff_list_t0, t= ts1 , args=(real_Mtensor,))
plt.figure(figsize=(7, 4))

for i in range(len(total_no_spins)):
    plt.plot(ts1, result[:,i], label=r'$c_{(%d)}(t)$'%i)

plt.xlabel("t")
plt.ylabel("Phi(t)")
plt.legend()

